** IMPORTS **

In [1]:
# # https://deeplearningcourses.com/c/deep-learning-advanced-nlp
# get the data at: http://www.manythings.org/anki/
from __future__ import print_function, division
from builtins import range, input
# Note: you may need to update your version of future
# sudo pip install -U future

import os, sys

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as display

import keras.backend as K
if len(K.tensorflow_backend._get_available_gpus()) > 0:
  from keras.layers import CuDNNLSTM as LSTM
  from keras.layers import CuDNNGRU as GRU
from keras.callbacks import Callback
from keras.callbacks import LearningRateScheduler
from keras.models import load_model
from keras import regularizers


import math
from numpy.random import seed
seed(9012)

Using TensorFlow backend.


In [2]:
import gc
gc.collect()

22

In [0]:
# SELECT THE MODEL OR LEAVE THE STRING EMPTY WITH ""
model_selected = "model_20191117-233555.h5"

dic_model = {"model_20191116-012401.h5" :{  "BATCH_SIZE" : 64,
                                            "EPOCHS" : 45,   
                                            "LATENT_DIM" : 256 ,
                                            "NUM_SAMPLES": 20000  ,
                                            "MAX_SEQUENCE_LENGTH" : 10,
                                            "MAX_NUM_WORDS" : 20000,
                                            "EMBEDDING_DIM" : 100,
                                            "SUBREDDIT" : None,
                                            "LR_DECAY": False,
                                            "GLOVE_LINK": "https://drive.google.com/open?id=1ykKSKPRB5N1AoLntB3SUyVSP7-z5xluM", #glove.twitter.27B.100d.txt
                                            "LINK" : "https://drive.google.com/open?id=1ixxHuOe185FJNwHqE_FYx67k1jQdotzR"},
             "model_20191117-191352.h5" :{  "BATCH_SIZE" : 32,
                                            "EPOCHS" : 20,   
                                            "LATENT_DIM" : 128 ,
                                            "NUM_SAMPLES" : 20000  ,
                                            "MAX_SEQUENCE_LENGTH" : 10,
                                            "MAX_NUM_WORDS" : 20000,
                                            "EMBEDDING_DIM" : 100,
                                            "SUBREDDIT" : None,
                                            "LR_DECAY": False,
                                            "GLOVE_LINK": "https://drive.google.com/open?id=1ykKSKPRB5N1AoLntB3SUyVSP7-z5xluM", #glove.twitter.27B.100d.txt
                                            "LINK" : "https://drive.google.com/open?id=1EZsk2nQs5T1uGSpIHkycWYqcNsI2_vDt"},
              "model_20191117-233555.h5" :{  "BATCH_SIZE" : 64,
                                              "EPOCHS" : 45,   
                                              "LATENT_DIM" : 256 ,
                                              "NUM_SAMPLES": 20000  ,
                                              "MAX_SEQUENCE_LENGTH" : 10,
                                              "MAX_NUM_WORDS" : 20000,
                                              "EMBEDDING_DIM" : 100,
                                              "SUBREDDIT" : None,
                                              "LR_DECAY": True, # 0.9
                                            "GLOVE_LINK": "https://drive.google.com/open?id=1ykKSKPRB5N1AoLntB3SUyVSP7-z5xluM", #glove.twitter.27B.100d.txt
                                            "LINK" : "https://drive.google.com/open?id=13kyuAcoULzFx2Hy3KOmlSDthGHXgdVMp"},
              "model_20191121-014934.h5" :{  "BATCH_SIZE" : 64,
                          "EPOCHS" : 30,   
                          "LATENT_DIM" : 256 ,
                          "NUM_SAMPLES": 20000  ,
                          "MAX_SEQUENCE_LENGTH" : 10,
                          "MAX_NUM_WORDS" : 20000,
                          "EMBEDDING_DIM" : 200,
                          "SUBREDDIT" : None,
                          "LR_DECAY": True,
                          "BIDIRECTIONAL" : False,
                                            "GLOVE_LINK": "https://drive.google.com/open?id=1V7tEAWJoLdl9Ul6h_glZqch5BjcU8Tqf", #glove.twitter.27B.200d.txt
                                            "LINK" : "https://drive.google.com/open?id=1yzzZEAiHx-CH_pFTsPWTt379hBaIxTfD"},
              "model_20191120-235108.h5" :{  "BATCH_SIZE" : 64,
                                              "EPOCHS" : 45,   
                                              "LATENT_DIM" : 256 ,
                                              "NUM_SAMPLES": 20000  ,
                                              "MAX_SEQUENCE_LENGTH" : 10,
                                              "MAX_NUM_WORDS" : 20000,
                                              "EMBEDDING_DIM" : 300,
                                              "SUBREDDIT" : None,
                                              "LR_DECAY": True,
                                            "GLOVE_LINK": "https://drive.google.com/open?id=1W9FeIDZcbUPFzk58bZ4Yw9nj2FX90OjJ", #glove.6B.300d.txt
                                            "LINK" : "https://drive.google.com/open?id=1kTJrPal6ioJS6wDuWAdaIOgJGXSCNYeb"},
             "model_20191121-160547.h5" :{   "BATCH_SIZE" : 64,
                          "EPOCHS" : 30,   
                          "LATENT_DIM" : 128 ,
                          "NUM_SAMPLES": 20000  ,
                          "MAX_SEQUENCE_LENGTH" : 10,
                          "MAX_NUM_WORDS" : 20000,
                          "EMBEDDING_DIM" : 200,
                          "SUBREDDIT" : None,
                          "LR_DECAY": True,#0.95
                          "BIDIRECTIONAL" : False,
                                            "GLOVE_LINK": "https://drive.google.com/open?id=1W9FeIDZcbUPFzk58bZ4Yw9nj2FX90OjJ", #glove.6B.300d.txt
                                            "LINK" : "https://drive.google.com/open?id=1kTJrPal6ioJS6wDuWAdaIOgJGXSCNYeb"},
             }
selected_dic_model = dic_model.get(model_selected)
if not model_selected:
  selected_dic_model = {   "BATCH_SIZE" : 64,
                          "EPOCHS" : 45,   
                          "LATENT_DIM" : 256 ,
                          "NUM_SAMPLES": 20000  ,
                          "MAX_SEQUENCE_LENGTH" : 10,
                          "MAX_NUM_WORDS" : 20000,
                          "EMBEDDING_DIM" : 100,
                          "SUBREDDIT" : None,
                          "LR_DECAY": True, # 0.9
                            "RNN": "LSTM",
                          "GLOVE_LINK": "https://drive.google.com/open?id=1ykKSKPRB5N1AoLntB3SUyVSP7-z5xluM",
                          "LINK" : None}


# some config
BATCH_SIZE = selected_dic_model.get("BATCH_SIZE")
EPOCHS = selected_dic_model.get("EPOCHS")
LATENT_DIM = selected_dic_model.get("LATENT_DIM")
NUM_SAMPLES = selected_dic_model.get("NUM_SAMPLES")
MAX_SEQUENCE_LENGTH = selected_dic_model.get("MAX_SEQUENCE_LENGTH")
MAX_NUM_WORDS = selected_dic_model.get("MAX_NUM_WORDS")
EMBEDDING_DIM = selected_dic_model.get("EMBEDDING_DIM")


In [4]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
link = "https://drive.google.com/open?id=1uH4pJhT7SX6lz-IALta41WTBJ1Y_k7oS"
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('sarcasm_only.csv') 

1uH4pJhT7SX6lz-IALta41WTBJ1Y_k7oS


In [0]:


# load in the data
data = pd.read_csv('sarcasm_only.csv',dtype=str)
data_clean = data[data['label'] == "1"]
data_clean = data_clean.astype(str)
data_clean.drop('ups', axis=1, inplace=True)
data_clean.drop('downs', axis=1, inplace=True)
data_clean.drop('Unnamed: 0', axis=1, inplace=True)
data_clean.drop('author', axis=1, inplace=True)
data_clean.drop('created_utc', axis=1, inplace=True)
data_clean.drop('date', axis=1, inplace=True)
data_clean.drop('label', axis=1, inplace=True)

def lower(x):
  return(x.lower())
data_clean['comment'] = data_clean.apply(lambda x: lower(x["comment"]), axis=1)
data_clean['parent_comment'] = data_clean.apply(lambda x: lower(x["parent_comment"]), axis=1)


In [7]:
co = data_clean["comment"].str.contains("the downvotes are because you ",na=False)
data_clean[co]["comment"].values

array(["hahahaha oh man, i'm guessing the downvotes are because you didn't end that with"],
      dtype=object)

In [0]:
appos = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not"
}

In [9]:
#Negation handling
def reform(sentence):
    words = sentence.split()
    reformed = [appos[word] if word in appos else word for word in words]
    reformed = " ".join(reformed) 
    return(reformed)
data_clean['comment'] = data_clean.apply(lambda x: reform(x["comment"]), axis=1)
data_clean['parent_comment'] = data_clean.apply(lambda x: reform(x["parent_comment"]), axis=1)
data_clean.head()

,comment,subreddit,score,parent_comment
0,but they will have all those reviews!,ProductTesting,0,"the dumb thing is, they are risking their sell..."
1,wow it is totally unreasonable to assume that ...,politics,2,clinton campaign accuses fbi of 'blatant doubl...
2,ho ho ho... but melania said that there is no ...,politics,8,anyone else think that it was interesting the ...
3,i cannot wait until @potus starts a twitter wa...,politics,3,here's what happens when obama gives up his tw...
4,gotta love the teachers who give exams on the ...,CFBOffTopic,3,monday night drinking thread brought to you by...


In [0]:

def count(x):
  return len(x.split())
data_clean['words_comment'] = data_clean.apply(lambda x: count(x["comment"]), axis=1)
data_clean['words_parent_comment'] = data_clean.apply(lambda x: count(x["parent_comment"]), axis=1)


In [11]:
co = data_clean["comment"].str.contains("the downvotes are because you ",na=False)
data_clean[co]["comment"].values

array(['hahahaha oh man, I am guessing the downvotes are because you did not end that with'],
      dtype=object)

In [12]:
short_sentences = (data_clean['words_comment'] <= selected_dic_model.get("MAX_SEQUENCE_LENGTH")) & (data_clean['words_parent_comment'] <= selected_dic_model.get("MAX_SEQUENCE_LENGTH"))
(data_clean[short_sentences].info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117710 entries, 5 to 505406
Data columns (total 6 columns):
comment                 117710 non-null object
subreddit               117710 non-null object
score                   117710 non-null object
parent_comment          117710 non-null object
words_comment           117710 non-null int64
words_parent_comment    117710 non-null int64
dtypes: int64(2), object(4)
memory usage: 6.3+ MB


In [13]:
# Where we will store the data
input_texts = [] # sentence in original language
target_texts = [] # sentence in target language
target_texts_inputs = [] # sentence in target language offset by 1
if selected_dic_model.get("SUBREDDIT") is not None:
  grouped = data_clean[short_sentences].groupby('subreddit')
  for subreddit_name, df_sub in grouped:
      if subreddit_name == "politics":
          for index, row in df_sub.iterrows():
              # recall we'll be using teacher forcing
              target_text = row['comment'] + ' <eos>'
              target_text_input = '<sos> ' + row['comment']
                    
              input_texts.append(row['parent_comment'])
              target_texts.append(target_text)
              target_texts_inputs.append(target_text_input)
  print("num samples:", len(input_texts))

else :
  data_clean = data_clean[short_sentences].sort_values('score',ascending = False).head(NUM_SAMPLES)
  for index, row in data_clean.iterrows():
      # recall we'll be using teacher forcing
      target_text = row['comment'] + ' <eos>'
      target_text_input = '<sos> ' + row['comment']
            
      input_texts.append(row['parent_comment'])
      target_texts.append(target_text)
      target_texts_inputs.append(target_text_input)
  print("num samples:", len(input_texts))

num samples: 20000


In [0]:
# tokenize the inputs
tokenizer_inputs = Tokenizer()
tokenizer_inputs.fit_on_texts(input_texts)
input_sequences = tokenizer_inputs.texts_to_sequences(input_texts)


In [15]:
# get the word to index mapping for input language
word2idx_inputs = tokenizer_inputs.word_index
print("%s mots (tokens) uniques à l'entrée" % len(word2idx_inputs))

# determine maximum length input sequence
max_len_input = max(len(s) for s in input_sequences)
print("La phrase la plus longue contient %s mots à l'entrée" % max_len_input)

19815 mots (tokens) uniques à l'entrée
La phrase la plus longue contient 21 mots à l'entrée


In [0]:

# tokenize the outputs
# don't filter out special characters
# otherwise <sos> and <eos> won't appear
tokenizer_outputs = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
tokenizer_outputs.fit_on_texts(target_texts + target_texts_inputs) # inefficient, oh well
target_sequences = tokenizer_outputs.texts_to_sequences(target_texts)
target_sequences_inputs = tokenizer_outputs.texts_to_sequences(target_texts_inputs)

In [17]:
# get the word to index mapping for output language
word2idx_outputs = tokenizer_outputs.word_index
print("%s mots (tokens) uniques à la sortie" % len(word2idx_outputs))
# store number of output words for later
# remember to add 1 since indexing starts at 1
num_words_output = len(word2idx_outputs) + 1

# determine maximum length output sequence
max_len_target = max(len(s) for s in target_sequences)
print("La phrase la plus longue contient %s mots à la sortie" % max_len_target)

24342 mots (tokens) uniques à la sortie
La phrase la plus longue contient 11 mots à la sortie


In [18]:
# pad the sequences
encoder_inputs = pad_sequences(input_sequences, maxlen=max_len_input)
print("encoder_inputs.shape:", encoder_inputs.shape)
print("encoder_inputs[0]:", encoder_inputs[0])

decoder_inputs = pad_sequences(target_sequences_inputs, maxlen=max_len_target, padding='post')
print("decoder_inputs[0]:", decoder_inputs[0])
print("decoder_inputs.shape:", decoder_inputs.shape)

decoder_targets = pad_sequences(target_sequences, maxlen=max_len_target, padding='post')




encoder_inputs.shape: (20000, 21)
encoder_inputs[0]: [   0    0    0    0    0    0    0    0    0    0    0    0    0 1013
  420   38  184    3  292   40  240]
decoder_inputs[0]: [   2  764 7319    0    0    0    0    0    0    0    0]
decoder_inputs.shape: (20000, 11)


In [19]:
link = selected_dic_model.get("GLOVE_LINK")
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('glove.txt') 

1ykKSKPRB5N1AoLntB3SUyVSP7-z5xluM


In [20]:
# store all the pre-trained word vectors
print('Loading word vectors...')
word2vec = {}
with open('glove.txt',encoding="utf8")as f:
  # is just a space-separated text file in the format:
  # word vec[0] vec[1] vec[2] ...
  for line in f:
    values = line.split()
    word = values[0]
    vec = np.asarray(values[1:], dtype='float32')
    word2vec[word] = vec
print('Found %s word vectors.' % len(word2vec))





Loading word vectors...
Found 1193514 word vectors.


In [21]:
# prepare embedding matrix
print('Filling pre-trained embeddings...')
num_words = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word2idx_inputs.items():
  if i < MAX_NUM_WORDS:
    embedding_vector = word2vec.get(word)
    if embedding_vector is not None:
      # words not found in embedding index will be all zeros.
      embedding_matrix[i] = embedding_vector

# create embedding layer
embedding_layer = Embedding(
  num_words,
  EMBEDDING_DIM,
  weights=[embedding_matrix],
  input_length=max_len_input,
  #trainable=True
)


# create targets, since we cannot use sparse
# categorical cross entropy when we have sequences
decoder_targets_one_hot = np.zeros(
  (
    len(input_texts),
    max_len_target,
    num_words_output
  ),
  dtype='float32'
)

# assign the values
for i, d in enumerate(decoder_targets):
  for t, word in enumerate(d):
    decoder_targets_one_hot[i, t, word] = 1



Filling pre-trained embeddings...



In [0]:
# Matplotlib config
plt.ioff()
plt.rc('image', cmap='gray_r')
plt.rc('grid', linewidth=1)
plt.rc('xtick', top=False, bottom=False, labelsize='large')
plt.rc('ytick', left=False, right=False, labelsize='large')
plt.rc('axes', facecolor='F8F8F8', titlesize="large", edgecolor='white')
plt.rc('text', color='a8151a')
plt.rc('figure', facecolor='F0F0F0', figsize=(16,9))

def plot_learning_rate(lr_func, epochs):
  xx = np.arange(epochs+1, dtype=np.float)
  y = [lr_decay(x) for x in xx]
  fig, ax = plt.subplots(figsize=(9, 6))
  ax.set_xlabel('epochs')
  ax.set_title('Learning rate\ndecays from {:0.3g} to {:0.3g}'.format(y[0], y[-2]))
  ax.minorticks_on()
  ax.grid(True, which='major', axis='both', linestyle='-', linewidth=1)
  ax.grid(True, which='minor', axis='both', linestyle=':', linewidth=0.5)
  ax.step(xx,y, linewidth=3, where='post')
  display.display(fig)

class LogThirdLayerOutput(Callback):
  def on_epoch_end(self, epoch, logs=None):
        layer_output = get_layer_output(self.decoder_targets_one_hot)[0]
        print(layer_output.shape)

class PlotTraining(Callback):
  def __init__(self, sample_rate=1, zoom=0.5):
    self.sample_rate = sample_rate
    self.step = 0
    self.zoom = zoom
    self.steps_per_epoch = NUM_SAMPLES//BATCH_SIZE

  def on_train_begin(self, logs={}):
    self.batch_history = {}
    self.batch_step = []
    self.epoch_history = {}
    self.epoch_step = []
    self.fig, self.axes = plt.subplots(1, 2, figsize=(16, 7))
    plt.ioff()

  def on_batch_end(self, batch, logs={}):
    if (batch % self.sample_rate) == 0:
      self.batch_step.append(self.step)
      for k,v in logs.items():
        # do not log "batch" and "size" metrics that do not change
        # do not log training accuracy "acc"
        if k=='batch' or k=='size':# or k=='acc':
          continue
        self.batch_history.setdefault(k, []).append(v)
    self.step += 1

  def on_batch_end(self, batch, logs={}):
    if (batch % self.sample_rate) == 0:
      self.batch_step.append(self.step)
      for k,v in logs.items():
        # do not log "batch" and "size" metrics that do not change
        # do not log training accuracy "acc"
        if k=='batch' or k=='size':# or k=='acc':
          continue
        self.batch_history.setdefault(k, []).append(v)
    self.step += 1

  def on_epoch_end(self, epoch, logs={}):
    plt.close(self.fig)
    self.axes[0].cla()
    self.axes[1].cla()
      
    self.axes[0].set_ylim(0, 5/self.zoom)
    self.axes[1].set_ylim(0.3, 1+0.1/self.zoom/2)
    
    self.epoch_step.append(self.step)
    for k,v in logs.items():
      # only log validation metrics
      if not k.startswith('val_'):
        continue
      self.epoch_history.setdefault(k, []).append(v)

    display.clear_output(wait=True)
    
    for k,v in self.batch_history.items():
      self.axes[0 if k.endswith('loss') else 1].plot(np.array(self.batch_step) / self.steps_per_epoch, v, label=k)
      
    for k,v in self.epoch_history.items():
      self.axes[0 if k.endswith('loss') else 1].plot(np.array(self.epoch_step) / self.steps_per_epoch, v, label=k, linewidth=3)
      
    self.axes[0].legend()
    self.axes[1].legend()
    self.axes[0].set_xlabel('epochs')
    self.axes[1].set_xlabel('epochs')
    self.axes[0].minorticks_on()
    self.axes[0].grid(True, which='major', axis='both', linestyle='-', linewidth=1)
    self.axes[0].grid(True, which='minor', axis='both', linestyle=':', linewidth=0.5)
    self.axes[1].minorticks_on()
    self.axes[1].grid(True, which='major', axis='both', linestyle='-', linewidth=1)
    self.axes[1].grid(True, which='minor', axis='both', linestyle=':', linewidth=0.5)
    display.display(self.fig)
    
plot_training = PlotTraining(sample_rate=10, zoom=1)

In [23]:

link = selected_dic_model.get("LINK")
fluff, model_id = link.split('=')
print (model_id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':model_id}) 
downloaded.GetContentFile('model.h5')
# load model
model = load_model('model.h5')


13kyuAcoULzFx2Hy3KOmlSDthGHXgdVMp






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [24]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 21)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 11)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 21, 100)      1977800     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 11, 256)      6260992     input_2[0][0]                    
____________________________________________________________________________________________

In [25]:
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping

EPOCHS=100
dropout = 0.5
regul = 0.01
#optim = selected_dic_model.get("OPTIMIZER")
#optim = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
#optim = optimizers.RMSprop(lr=0.001, rho=0.9)
optim = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999)
if selected_dic_model.get("LINK") is None: 
  if selected_dic_model.get("RNN") == "LSTM":
    ##### build the model #####
    encoder_inputs_placeholder = Input(shape=(max_len_input,))
    x = embedding_layer(encoder_inputs_placeholder)
    encoder = LSTM(
      LATENT_DIM,
      return_state=True,
      dropout=dropout, # dropout not available on gpu,
      recurrent_regularizer=regularizers.l2(regul)
    )
    encoder_outputs, h, c = encoder(x)
    # encoder_outputs, h = encoder(x) #gru

    # keep only the states to pass into decoder
    encoder_states = [h, c]

  elif selected_dic_model.get("RNN") == "GRU":
    encoder_inputs_placeholder = Input(shape=(max_len_input,))
    x = embedding_layer(encoder_inputs_placeholder)
    encoder = GRU(
      LATENT_DIM,
      return_state=True,
      recurrent_regularizer=regularizers.l2(regul)
    )
    encoder_outputs, h = encoder(x) #gru

    # keep only the states to pass into decoder
    encoder_states = [h]

  decoder_inputs_placeholder = Input(shape=(max_len_target,))

  # this word embedding will not use pre-trained vectors
  # although you could
  decoder_embedding = Embedding(num_words_output, LATENT_DIM)
  decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)
  if selected_dic_model.get("RNN") == "LSTM":
    # since the decoder is a "to-many" model we want to have
    # return_sequences=True
    decoder_lstm = LSTM(
      LATENT_DIM,
      return_sequences=True,
      return_state=True,
      recurrent_regularizer=regularizers.l2(regul)

    )
    decoder_outputs, _, _ = decoder_lstm(
      decoder_inputs_x,
      initial_state=encoder_states
    )
  elif selected_dic_model.get("RNN") == "GRU":
    # since the decoder is a "to-many" model we want to have
    # return_sequences=True
    decoder_gru = GRU(
      LATENT_DIM,
      return_sequences=True,
      return_state=True,
      dropout=dropout, # dropout not available on gpu
      recurrent_regularizer=regularizers.l2(regul)

    )
    decoder_outputs, _ = decoder_gru(
      decoder_inputs_x,
      initial_state=encoder_states
    )

  # final dense layer for predictions
  decoder_dense = Dense(num_words_output, activation='softmax')
  decoder_outputs = decoder_dense(decoder_outputs)

  # Create the model object
  model = Model([encoder_inputs_placeholder, decoder_inputs_placeholder], decoder_outputs)

  # Compile the model and train it
  model.compile(
    optimizer=optim,
    loss='categorical_crossentropy',
    metrics=['accuracy'])
  early_stopping = EarlyStopping(monitor='val_loss', patience=5)

  r = model.fit(
    [encoder_inputs, decoder_inputs], decoder_targets_one_hot,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.2, 
    callbacks=[plot_training,early_stopping])

  # plot some data
  plt.plot(r.history['loss'], label='loss')
  plt.plot(r.history['val_loss'], label='val_loss')
  plt.legend()
  plt.show()
  # accuracies
  plt.plot(r.history['acc'], label='acc')
  plt.plot(r.history['val_acc'], label='val_acc')
  plt.legend()
  plt.show()
  import time

  timestr = time.strftime("%Y%m%d-%H%M%S")
  filename = 'model_%s.h5' % timestr
  filename_r = 'model_%s.csv' % timestr

  # Save model
  model.save(filename)
  model_file = drive.CreateFile({'title' : filename})
  model_file.SetContentFile(filename)  
  model_file.Upload()
  drive.CreateFile({'id': model_file.get('id')})

  import pandas as pd
  # Save model
  hist_df = pd.DataFrame(r.history) 
  hist_df.to_csv(filename_r)
  model_file_r = drive.CreateFile({'title' : filename_r})
  model_file_r.SetContentFile(filename_r)  
  model_file_r.Upload()
  drive.CreateFile({'id': model_file_r.get('id')})
else:
  link = selected_dic_model.get("LINK")
  fluff, model_id = link.split('=')
  print (model_id) # Verify that you have everything after '='
  downloaded = drive.CreateFile({'id':model_id}) 
  downloaded.GetContentFile('model.h5')
  # load model
  model = load_model('model.h5')
  encoder_inputs_placeholder = model.get_layer(name="input_9").input
  decoder_inputs_placeholder = model.get_layer(name="input_10").input

  encoder_lstm = model.get_layer(name="lstm_3")
  decoder_lstm = model.get_layer(name="lstm_4")


  encoder_embedding = model.get_layer("embedding_1")
  decoder_embedding = model.get_layer("embedding_3")

  decoder_dense = model.get_layer("dense_2")

  encoder_outputs, h, c = encoder_lstm(encoder_embedding.output)
  encoder_states = [h, c]



13kyuAcoULzFx2Hy3KOmlSDthGHXgdVMp


ValueError: ignored

In [0]:
  # Save model
  model.save(filename)
  model_file = drive.CreateFile({'title' : filename})
  model_file.SetContentFile(filename)  
  model_file.Upload()
  drive.CreateFile({'id': model_file.get('id')})
  
  import pandas as pd
  # Save model
  hist_df = pd.DataFrame(r.history) 
  hist_df.to_csv(filename_r)
  model_file_r = drive.CreateFile({'title' : filename_r})
  model_file_r.SetContentFile(filename_r)  
  model_file_r.Upload()
  drive.CreateFile({'id': model_file_r.get('id')})

In [0]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [0]:
##### Make predictions #####
# As with the poetry example, we need to create another model
# that can take in the RNN state and previous word as input
# and accept a T=1 sequence.

# The encoder will be stand-alone
# From this we will get our initial decoder hidden state
encoder_model = Model(encoder_inputs_placeholder, encoder_states)


if selected_dic_model.get("RNN") == "LSTM":
  decoder_state_input_h = Input(shape=(LATENT_DIM,))
  decoder_state_input_c = Input(shape=(LATENT_DIM,))  
  decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
elif selected_dic_model.get("RNN") == "GRU":
  decoder_state_input_h = Input(shape=(LATENT_DIM,))
  decoder_states_inputs = [decoder_state_input_h] # gru

decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

In [0]:


# this time, we want to keep the states too, to be output
# by our sampling model
if selected_dic_model.get("RNN") == "LSTM":
  decoder_outputs, h, c = decoder_lstm(
    decoder_inputs_single_x,
    initial_state=decoder_states_inputs)
  decoder_states = [h, c]

elif selected_dic_model.get("RNN") == "GRU":
  decoder_outputs, h = decoder_gru(
    decoder_inputs_single_x,
    initial_state=decoder_states_inputs )
  decoder_states = [h] # gru


decoder_outputs = decoder_dense(decoder_outputs)


In [0]:

# The sampling model
# inputs: y(t-1), h(t-1), c(t-1)
# outputs: y(t), h(t), c(t)

decoder_model = Model([decoder_inputs_single] + decoder_states_inputs,[decoder_outputs]+ decoder_states)

# map indexes back into real words
# so we can view the results
idx2word_eng = {v:k for k, v in word2idx_inputs.items()}
idx2word_trans = {v:k for k, v in word2idx_outputs.items()}

In [0]:

def decode_sequence(input_seq):
  # Encode the input as state vectors.
  states_value = encoder_model.predict(input_seq)
  # Generate empty target sequence of length 1.
  target_seq = np.zeros((1, 1))

  # Populate the first character of target sequence with the start character.
  # NOTE: tokenizer lower-cases all words
  target_seq[0, 0] = word2idx_outputs['<sos>']


  # if we get this we break
  eos = word2idx_outputs['<eos>']

  # Create the translation
  output_sentence = []
  for _ in range(max_len_target):
    if selected_dic_model.get("RNN") == "LSTM":
      output_tokens, h, c = decoder_model.predict(
        [target_seq] + states_value
      )
    elif selected_dic_model.get("RNN") == "GRU":
      output_tokens, h = decoder_model.predict(
      [target_seq] + states_value ) # gru

    # Get next word
    idx = np.argmax(output_tokens[0, 0, :])

    # End sentence of EOS
    if eos == idx:
      break

    word = ''
    if idx > 0:
      word = idx2word_trans[idx]
      output_sentence.append(word)

    # Update the decoder input
    # which is just the word just generated
    target_seq[0, 0] = idx

    # Update states
    if selected_dic_model.get("RNN") == "LSTM":
      states_value = [h, c]
    elif selected_dic_model.get("RNN") == "GRU":
      states_value = [h] # gru

  return ' '.join(output_sentence)





In [0]:
sentences_5 = ["He is the best",
             "Should have gotten a bike",
             "he doesnt really speak english",
             "Good luck penetrating German airspace",
             "YouTube is not being honest",
             "Preparing a taco in space",
             "Is that a legal solution",
             "School is really hard",
             "Today was a good day",
             "CNN and their debates"]
for sentence in sentences_5 :           
  input_seq = tokenizer_inputs.texts_to_sequences(sentence.split())
  print(input_seq)
  input_seq_new = np.array(input_seq, ndmin=1).transpose() 
  encoder_inputs_fake = pad_sequences(input_seq_new, maxlen=max_len_input)
  translation = decode_sequence(encoder_inputs_fake)
  states_value = encoder_model.predict(encoder_inputs_fake)
  print('Input: "{}":       Sarcasm: "{}"'.format(sentence,translation))




In [0]:
word = " downvotes are because you"
comment =data_clean["comment"].str.contains(word) 
parent_comment =data_clean["parent_comment"].str.contains(word) 

print(data_clean[comment])
print(data_clean[parent_comment])